In [31]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import connectDB
import pandas as pd
import plotly.graph_objects as go

# DB 연결
myclient = connectDB.connectDB()

myDB = myclient['FriDB']
newDB = myclient['newDB']
newCollection = newDB['DocPopData']


popDF = pd.read_csv('data/추계인구.csv', encoding='cp949')

popDF.replace('-',0)

,행정구분,성별,년도,값
0,서울,계,1970,5685932
1,서울,계,1971,5940442
2,서울,계,1972,6206625
3,서울,계,1973,6470497
4,서울,계,1974,6737832
...,...,...,...,...
3883,제주,여자,2043,392382
3884,제주,여자,2044,392503
3885,제주,여자,2045,392376
3886,제주,여자,2046,391993


In [32]:
popDF.groupby(['년도','성별','행정구분']).sum()

값
년도   성별 행정구분         
1970 계  강원    1913786
        경기    2636096
        경남    2794945
        경북    3373973
        광주     651380
...               ...
2047 여자 전남     797578
        전북     796979
        제주     391333
        충남    1127722
        충북     800008

[3888 rows x 1 columns]

In [33]:
docDataDF = pd.DataFrame(newCollection.find({},{'_id':0}))
docDataDF[(docDataDF.행정구분 == '세종')]

,행정구분,년도,직무구분,데이터구분,값
608,세종,2009,의사,인원수,0.0
657,세종,2012,의사,인원수,79.0
674,세종,2013,의사,인원수,97.0
691,세종,2014,의사,인원수,147.0
708,세종,2015,의사,인원수,152.0
725,세종,2016,의사,인원수,171.0
1405,세종,2012,한의사,인원수,21.0
1422,세종,2013,한의사,인원수,34.0
1439,세종,2014,한의사,인원수,35.0
1456,세종,2015,한의사,인원수,46.0


In [34]:
docDataDF[(docDataDF.년도 == 2001)&(docDataDF.행정구분!='전국')]

,행정구분,년도,직무구분,데이터구분,값
464,강원,2001,의사,인원수,1617.0
465,경기,2001,의사,인원수,8102.0
466,경남,2001,의사,인원수,2480.0
467,경북,2001,의사,인원수,1862.0
468,광주,2001,의사,인원수,2139.0
469,대구,2001,의사,인원수,3586.0
470,대전,2001,의사,인원수,2023.0
471,부산,2001,의사,인원수,4837.0
472,서울,2001,의사,인원수,18097.0
473,울산,2001,의사,인원수,884.0


In [43]:
resultList = []
for year in range(1970, 2017):
    dataDF = docDataDF[(docDataDF.년도 == year)&(docDataDF.행정구분!='전국')&(docDataDF.행정구분!='합계')]
    if len(dataDF)>0:
        #print('len :', len(dataDF))
        for i in range(len(dataDF)):
            local = dataDF.iloc[i,0]
            value = dataDF.iloc[i,4]
            popYear = popDF[(popDF.성별 == '계')&(popDF.년도==year)&(popDF.행정구분==local)].iloc[0,3]
            docType = dataDF.iloc[i,2]
            if float(popYear)>0:
                perValue = round(float(value)/float(popYear)*1000,3)
                resultDict = {'행정구분':local,'년도':year,'직무구분':docType,'데이터구분':'천명당인원수','값':perValue}
                #print(resultDict)
                resultList.append(resultDict)

resultList

[{'행정구분': '강원', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '경기', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '경남', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '경북', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '광주', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '대구', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '대전', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '부산', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.427},
 {'행정구분': '서울', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.713},
 {'행정구분': '울산', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '인천', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '전남', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '전북', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수', '값': 0.0},
 {'행정구분': '제주', '년도': 1970, '직무구분': '의사', '데이터구분': '천명당인원수',

In [44]:
newCollection.insert_many(resultList)